In [3]:
###########################################
PROJECT_NAME = 'wheel_axle'
###########################################

In [3]:
import config #place your Openai key here
from openai import OpenAI

# Initialize the OpenAI client with API key
client = OpenAI(api_key=config.OPENAI_API_KEY)

In [4]:
# Global variables to keep track of the conversation
conversation_history = []
last_response = None
model_call_count = 0

# Compose the system message once and append to history
with open("system_sif.txt", 'r') as file:
    sif_message = file.read().strip()
with open("mesh.names", 'r') as file:
    geometry_message = file.read().strip()

system_message = sif_message + geometry_message 
    
conversation_history.append({"role": "system", "content": system_message})

In [5]:
#FUNCTIONS FOR INTERACTING WITH CHATBOT
    
def chat_with_bot(user_messages):
    global last_response
    # Add the new user input to the conversation history
    conversation_history.append({"role": "user", "content": user_messages})
    # Call the OpenAI model
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=conversation_history,
        temperature=0
    )
    # Extract the model's response
    model_response = response.choices[0].message.content
    # Update the last response
    last_response = model_response
    # Append the model's response to the conversation history
    conversation_history.append({"role": "assistant", "content": model_response})
    # Print the model's response
    return print(model_response)

#Saves .geo file from response
def extract_and_save_sif_file(response_text): 
    file_path = f"{PROJECT_NAME}.sif"
    try:
        # Use string formatting explained in system_geo.txt 
        start_marker = "//BEGIN_SIF"
        end_marker = f"//END_SIF"

        start_index = response_text.index(start_marker) + len(start_marker)
        end_index = response_text.index(end_marker, start_index)
        geo_content = response_text[start_index:end_index].strip()

        with open(file_path, 'w') as file:
            file.write(geo_content)
        print(f"{file_path} file saved")

    except ValueError as e:
        print(f"ERROR: .geo file NOT SAVED.")

In [2]:
import os

def generate_ELMER_mesh():
    try:
        current_directory = os.getcwd()
        msh_file = os.path.join(current_directory, f'{PROJECT_NAME}.msh')  
        !ElmerGrid 14 2 "{msh_file}" -autoclean
        print(f"ELMER project created in {PROJECT_NAME}")
        print(f"Copy .sif file in {PROJECT_NAME}")
    except ValueError as e:
        print(f"ERROR.")

In [6]:
############ INSTRUCTIONS #######################
# Chat with model with chat_with_bot("..."), 
# when you are satisfied save the .sif file with
# extract_and_save_sif_file(last_response) and
# generate ELMER project with generate_ELMER_mesh()
# use Paraview for visualization
###############################################

In [ ]:
generate_ELMER_mesh()
# move the mesh.names in the working directory

In [7]:
chat_with_bot("""Create a sif file for a 'linear elasticity' simulation of the wheel and axle with the boundaries defined in the 
mesh file. Boundary Condition at the "Bottom" should be fixed (no deformation), while the Boudary Condition at the "Top" implies a force 
of 5000000000 N in the z direction""")

To create the .sif file for the 'linear elasticity' simulation of the wheel and axle, I will need to know the following details:

1. Simulation Type: Should the simulation be "Steady state" or "Transient"?
2. Material Properties: What are the material properties of the wheel and axle (e.g., Young's modulus, Poisson ratio, density)?
3. Body Properties: Are there any specific body forces or initial conditions that need to be applied to the "MainBody"?
4. Solver Configuration: Are there any specific solver settings or tolerances that you would like to use?
5. Output Instructions: What results are you interested in obtaining from the simulation (e.g., displacements, stresses), and do you have any specific instructions for the output format or directory?

Once I have this information, I can proceed to create the .sif file for you.


In [8]:
chat_with_bot("""
1. Simulation Type: "Steady state"
2. Material Properties of MainBody: use typical values for steel
3. No body forces
4. Use default ones solver settings.
5. Displacement
Also, for Initial condition, assume displacement = 0
""")

Based on the information provided, here is the .sif file for a 'linear elasticity' simulation of the wheel and axle with the specified boundary conditions and material properties for steel:

```plaintext
//BEGIN_SIF

Header
  CHECK KEYWORDS Warn
  Mesh DB "." "."
  Include Path ""
  Results Directory ""
End

Simulation
  Max Output Level = 1
  Coordinate System = Cartesian
  Coordinate Mapping(3) = 1 2 3
  Simulation Type = Steady state
  Steady State Max Iterations = 1
  Output Intervals(1) = 1
  Solver Input File = case.sif
  Post File = case.vtu
End

Constants
  Gravity(4) = 0 -1 0 9.82
  Stefan Boltzmann = 5.670374419e-08
  Permittivity of Vacuum = 8.85418781e-12
  Permeability of Vacuum = 1.25663706e-6
  Boltzmann Constant = 1.380649e-23
  Unit Charge = 1.6021766e-19
End

Body 1
  Target Bodies(1) = 1
  Name = "MainBody"
  Equation = 1
  Material = 1
  Initial condition = 1
End

Solver 1
  Equation = Linear elasticity
  Procedure = "StressSolve" "StressSolver"
  Exec Solver = Alwa

In [9]:
extract_and_save_sif_file(last_response)

wheel_axle.sif file saved


In [4]:
import os
new_directory = os.path.join(os.getcwd(), f"{PROJECT_NAME}")
os.chdir(new_directory)

In [5]:
!ElmerSolver wheel_axle.sif

ELMER SOLVER (v 9.0) STARTED AT: 2023/12/03 17:21:25
ParCommInit:  Initialize #PEs:            1
MAIN: 
MAIN: =============================================================
MAIN: ElmerSolver finite element software, Welcome!
MAIN: This program is free software licensed under (L)GPL
MAIN: Copyright 1st April 1995 - , CSC - IT Center for Science Ltd.
MAIN: Webpage http://www.csc.fi/elmer, Email elmeradm@csc.fi
MAIN: Version: 9.0 (Rev: Release, Compiled: 2022-08-02)
MAIN:  Running one task without MPI parallelization.
MAIN:  Running with just one thread per task.
MAIN:  Lua interpreter linked in.
MAIN: =============================================================
MAIN: 
MAIN: 
MAIN: -------------------------------------
MAIN: Reading Model: wheel_axle.sif
LoadInputFile: Scanning input file: wheel_axle.sif
LoadInputFile: Scanning only size info
LoadInputFile: First time visiting
LoadInputFile: Reading base load of sif file
LoadInputFile: Loading input file: wheel_axle.sif
LoadInputFile: Rea